# Libraries

In [ ]:
import os
import cv2
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import albumentations as A
import glob
import segmentation_models as sm
from skimage import measure
from sklearn.metrics import classification_report
sm.set_framework('tf.keras')
sm.framework()

Segmentation Models: using `keras` framework.


'tf.keras'

In [ ]:
import GPUtil
import psutil
import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"]="2"

GPUtil.showUtilization()


| ID | GPU | MEM |
------------------
|  0 |  0% | 17% |
|  1 |  0% | 17% |
|  2 |  0% |  0% |
|  3 |  0% |  0% |


# Read data

In [ ]:
DATA_DIR = './net2_data/'

x_train_dir = os.path.join(DATA_DIR, 'train_rgb_70/')
y_train_dir = os.path.join(DATA_DIR, 'train_anno_70/')

x_valid_dir = os.path.join(DATA_DIR, 'test_rgb_30/')
y_valid_dir = os.path.join(DATA_DIR, 'test_anno_30/')


In [ ]:

def visualize(**images):
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()


def denormalize(x):
    x_max = np.percentile(x, 98)
    x_min = np.percentile(x, 2)
    x = (x - x_min) / (x_max - x_min)
    x = x.clip(0, 1)
    return x


# classes for data loading and preprocessing
class Dataset:


    CLASSES = ['nonmanmade', 'manmade']

    def __init__(
            self,
            images_dir,
            masks_dir,
            classes=None,
            augmentation=None,
            preprocessing=None,
    ):
        self.ids = os.listdir(images_dir)
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]

        # convert str names to class values on masks
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]

        self.augmentation = augmentation
        self.preprocessing = preprocessing

    def __getitem__(self, i):

        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.masks_fps[i], 0)

        # extract certain classes from mask
        masks = [(mask == v) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype('float')

        # add background if mask is not binary
        if mask.shape[-1] != 1:
            background = 1 - mask.sum(axis=-1, keepdims=True)
            mask = np.concatenate((mask, background), axis=-1)

        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']

        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']

        return image, mask

    def __len__(self):
        return len(self.ids)


class Dataloder(tf.keras.utils.Sequence):

    def __init__(self, dataset, batch_size=1, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(dataset))

        self.on_epoch_end()

    def __getitem__(self, i):

        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        # transpose list of lists
        batch = [np.stack(samples, axis=0) for samples in zip(*data)]

        return batch

    def __len__(self):

        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):

        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)

In [ ]:
def round_clip_0_1(x, **kwargs):
    return x.round().clip(0, 1)

def get_training_augmentation():
    train_transform = [

        A.HorizontalFlip(p=0.7),
        A.VerticalFlip(p=0.7),


        A.Lambda(mask=round_clip_0_1)
    ]
    return A.Compose(train_transform)

def get_validation_augmentation():

    test_transform = [
        A.PadIfNeeded(384, 384)
    ]
    return A.Compose(test_transform)

def get_preprocessing(preprocessing_fn):


    _transform = [
        A.Lambda(image=preprocessing_fn),
    ]
    return A.Compose(_transform)

# Model training

In [ ]:
# define network parameters
BACKBONE = 'vgg16'
BATCH_SIZE = 8
CLASSES = [ 'manmade']
LR = 0.0001
EPOCHS = 80
n_classes = 1
activation = 'sigmoid'

preprocess_input = sm.get_preprocessing(BACKBONE)

#create model
model = sm.FPN(BACKBONE, classes=n_classes, activation=activation,encoder_weights='imagenet',pyramid_dropout=0.3)


from tensorflow import keras

optim = keras.optimizers.Adam(LR)

dice_loss = sm.losses.DiceLoss(class_weights=np.array([1,1]))
focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

model.compile(optim, total_loss, metrics)

model_name = './net2_fpn.h5'

In [ ]:

train_dataset = Dataset(
    x_train_dir,
    y_train_dir,
    classes=CLASSES,
    augmentation=get_training_augmentation(),
    preprocessing=get_preprocessing(preprocess_input),
)

# Dataset for validation images
valid_dataset = Dataset(
    x_valid_dir,
    y_valid_dir,
    classes=CLASSES,
    augmentation=get_validation_augmentation(),
    preprocessing=get_preprocessing(preprocess_input),
)

train_dataloader = Dataloder(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = Dataloder(valid_dataset, batch_size=1, shuffle=False)

assert train_dataloader[0][0].shape == (BATCH_SIZE,384, 384,3)
assert train_dataloader[0][1].shape == (BATCH_SIZE,384, 384, n_classes)


callbacks = [
    keras.callbacks.ModelCheckpoint(model_name , save_weights_only=True, save_best_only=True, mode='min'),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7, verbose=True),
]

history = model.fit_generator(
    train_dataloader,
    steps_per_epoch=len(train_dataloader),
    epochs= 80,#EPOCHS,
    callbacks=callbacks,
    validation_data=valid_dataloader,
    validation_steps=len(valid_dataloader),
)


model.load_weights(model_name)

direction = glob.glob(x_valid_dir+'*.png')

dataset,d = valid_dataset, 'test_rgb_30/'
preds,annos = [], []
for i in range(len(dataset)):

    image, gt_mask = dataset[i]
    image = np.expand_dims(image, axis=0)
    pr_mask = model.predict(image).round()
    newimg = pr_mask[0,:,:,:]

    name = direction[i].replace('test_rgb_30','test_anno_30')

    preds.append(newimg[:,:,0])
    annos.append(cv2.imread(name)[:,:,0])
print('test classification_report ')
print(classification_report(np.array(annos).flatten(), np.array(preds).flatten(), labels=[0,1], digits = 4))
##################################################################
##################################################
direction = glob.glob(x_train_dir+'*.png')

dataset,d = train_dataset, 'train_rgb_70/'
preds,annos = [], []
for i in range(len(dataset)):

    image, gt_mask = dataset[i]
    image = np.expand_dims(image, axis=0)
    pr_mask = model.predict(image).round()

    newimg = pr_mask[0,:,:,:]
    name = direction[i].replace('train_rgb_70','train_anno_70')
    preds.append(newimg[:,:,0])
    annos.append(cv2.imread(name)[:,:,0])
print('train is saved')
print(classification_report(np.array(annos).flatten(), np.array(preds).flatten(), labels=[0,1], digits = 4))

/home/nayereh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/80
1032/1032 [==============================] - 712s 678ms/step - loss: 0.4450 - iou_score: 0.4920 - f1-score: 0.6440 - val_loss: 0.6796 - val_iou_score: 0.5579 - val_f1-score: 0.6341 - lr: 1.0000e-04
Epoch 2/80
1032/1032 [==============================] - 711s 689ms/step - loss: 0.3270 - iou_score: 0.5616 - f1-score: 0.7097 - val_loss: 0.6778 - val_iou_score: 0.6324 - val_f1-score: 0.7071 - lr: 1.0000e-04
Epoch 3/80
1032/1032 [==============================] - 718s 695ms/step - loss: 0.3067 - iou_score: 0.5821 - f1-score: 0.7269 - val_loss: 0.6626 - val_iou_score: 0.6342 - val_f1-score: 0.7076 - lr: 1.0000e-04
Epoch 4/80
1032/1032 [==============================] - 725s 703ms/step - loss: 0.2923 - iou_score: 0.5969 - f1-score: 0.7394 - val_loss: 0.6635 - val_iou_score: 0.6040 - val_f1-score: 0.6782 - lr: 1.0000e-04
Epoch 5/80
1032/1032 [==============================] - 725s 702ms/step - loss: 0.2836 - iou_score: 0.6062 - f1-score: 0.7472 - val_loss: 0.6646 - val_iou_score: 0.